In [1]:
import numpy as np
import pandas as pd
from importlib import reload
import datetime

import shared
import config
import provider_yfinance as provider
import model

reload(shared)
reload(config)
reload(provider)
reload(model)

cfg = config.get_config('^GDAXI')

# overwrite download_end_dt: use cached data
# config.overwrite_end_dt(cfg, '2019-12-19')
# config.save_config(cfg)

config> created config from file: './config.json'
config> config
        - base:
            - config_file_path: /mnt/c/notebooks/sandbox/config.json
        - datasets:
            - stocks: 30
            - benchmarks: 69
        - prepare:
            - data_start_dt: 2018-02-16
            - data_end_dt: 2020-01-16
            - cache_dir: /mnt/c/notebooks/sandbox/cache/20200116/
        - train:            
            - window_trading_days: [3, 5, 21, 35, 50]
            - lag_trading_days: [1, 2, 3, 4, 5]
            - label_max_high_weight: 3.0
            - label_max_close_weight: 1.0
            - settings: 12
        - model:
            - max_samples: 40
            - batch_size: 32
            - learning_rate: 0.1
            - learning_rate_decay: 0.75
            - lstm_hidden_size: 256
            - early_stopping_patience: 10
            - validation_monitor: val_mean_squared_error
            - max_epochs: 1000
            - base_dir: /mnt/c/notebooks/sandbox/model/20

In [2]:
cfg.model

Munch({'base_dir': '/mnt/c/notebooks/sandbox/model/20200116/', 'batch_size': 32, 'early_stopping_patience': 10, 'learning_rate': 0.1, 'learning_rate_decay': 0.75, 'lstm_hidden_size': 256, 'max_epochs': 1000, 'max_samples': 40, 'model_templates_dir': '/mnt/c/notebooks/sandbox/model/20200107', 'model_weights_file_name': 'model_weights.hdf5', 'optimizer_weights_file_name': 'optimizer_weights.pkl', 'validation_monitor': 'val_mean_squared_error'})

In [3]:
[(i, s.id) for i, s in enumerate(cfg.train.settings)]

[(0, 'lookback_3-label_1'),
 (1, 'lookback_3-label_2'),
 (2, 'lookback_5-label_1'),
 (3, 'lookback_5-label_3'),
 (4, 'lookback_7-label_1'),
 (5, 'lookback_7-label_3'),
 (6, 'lookback_10-label_3'),
 (7, 'lookback_14-label_3'),
 (8, 'lookback_21-label_5'),
 (9, 'lookback_36-label_7'),
 (10, 'lookback_64-label_7'),
 (11, 'lookback_100-label_7')]

In [4]:
%%time

model.train_full(cfg, 11)

sm-lookback_100-label_7> training submodel ...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 100, 1319)         5276      
_________________________________________________________________
masking (Masking)            (None, 100, 1319)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 100, 256)          1613824   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 2,669,981
Trainable params: 2,667,343
Non-trainable params: 2

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 100, 1319)         5276      
_________________________________________________________________
masking (Masking)            (None, 100, 1319)         0         
_________________________________________________________________
lstm (LSTM)                  (None, 100, 256)          1613824   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 2,669,981
Trainable params: 2,667,343
Non-trainable params: 2,638
__________________________________________

'/mnt/c/notebooks/sandbox/model/20200116/'

https://stackoverflow.com/questions/39674713/neural-network-lstm-input-shape-from-dataframe
https://stackoverflow.com/questions/49803503/lstm-preprocessing-build-3d-arrays-from-pandas-data-frame-based-on-id